In [1]:
import pandas as pd
# read the csv file
df=pd.read_csv('alldata.csv')
print(df.head)

<bound method NDFrame.head of        Unnamed: 0          Drug Name  Drug ID Cell Line Name  Cosmic ID  \
0               0       Camptothecin     1003           TE-5     735784   
1               1       Camptothecin     1003       EC-GI-10     753555   
2               2       Camptothecin     1003          HCE-4     753559   
3               3       Camptothecin     1003       KYSE-140     753573   
4               4       Camptothecin     1003       KYSE-410     753574   
...           ...                ...      ...            ...        ...   
90996        6091  N-acetyl cysteine     2499           FU97    1290806   
90997        6092  N-acetyl cysteine     2499         NUGC-4    1298357   
90998        6093  N-acetyl cysteine     2499         OCUM-1    1298358   
90999        6094  N-acetyl cysteine     2499         HSC-39    1322224   
91000        6095  N-acetyl cysteine     2499        SK-GT-2    1503364   

      TCGA Classification                Tissue Tissue Sub-type      

In [2]:
#check for null values
df.isnull().sum()
print(len(list(df['IC50'])))

91001


In [3]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
column_names=list(df.columns.values)
print(column_names)
data=df[['Drug Name', 'Cell Line Name', 'Tissue', 'Tissue Sub-type','IC50']]
data=data.dropna()
print(len(list(data['IC50'])))

['Unnamed: 0', 'Drug Name', 'Drug ID', 'Cell Line Name', 'Cosmic ID', 'TCGA Classification', 'Tissue', 'Tissue Sub-type', 'IC50', 'AUC', 'Max Conc', 'RMSE', 'Z score', 'Dataset Version']
91001


In [4]:
from collections import defaultdict
d = defaultdict(LabelEncoder)
df=data.dropna()
# Encoding the variable
fit = df.apply(lambda x: d[x.name].fit_transform(x))
fit=fit[['Drug Name','Cell Line Name','Tissue','Tissue Sub-type']]
X=fit[['Drug Name','Cell Line Name','Tissue','Tissue Sub-type']]
Y=data[['IC50']]
X,Y

(       Drug Name  Cell Line Name  Tissue  Tissue Sub-type
 0             73             853       0               42
 1             73             164       0               42
 2             73             253       0               42
 3             73             379       0               42
 4             73             384       0               42
 ...          ...             ...     ...              ...
 90996        174             203       4               50
 90997        174             631       4               50
 90998        174             644       4               50
 90999        174             273       4               50
 91000        174             757       4               50
 
 [91001 rows x 4 columns],
             IC50
 0      -2.555311
 1      -3.125664
 2      -3.536140
 3      -0.402974
 4       0.605862
 ...          ...
 90996  10.541691
 90997   9.745149
 90998  10.959875
 90999   8.342217
 91000  10.192296
 
 [91001 rows x 1 columns])

In [5]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
import numpy as np
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [6]:
D=Y_train[['IC50']]
d=np.array(list(Y_train['IC50']))
d

array([-1.80257327, -1.41593497,  1.55356364, ...,  5.0304007 ,
        2.54239184,  2.91726139])

In [29]:
#Random forest regression to predict IC50 based on the drug name, 
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 0)
regressor.fit(X_train, d)  
pred=regressor.predict(X_test)

In [30]:
print(regressor.score(X_test,Y_test))

0.7325508134586167


In [31]:
#compare predicted and actual IC50
pred,np.array(list(Y_test['IC50']))

(array([ 4.21450467, -1.25788903,  1.35734044, ...,  3.41185224,
         5.44768684,  3.53511919]),
 array([ 5.80561452, -1.62966712,  1.74246248, ...,  2.27048316,
         4.40686894,  4.23467973]))

In [14]:
Y_test,pd.DataFrame(pred)

(           IC50
 10761  5.805615
 15522 -1.629667
 918    1.742462
 69794  5.492629
 44075  5.703251
 ...         ...
 18622  0.060204
 83882  2.705581
 49765  2.270483
 7944   4.406869
 86328  4.234680
 
 [18201 rows x 1 columns],
               0
 0      4.208834
 1     -1.151270
 2      1.342674
 3      2.666075
 4      4.652400
 ...         ...
 18196  0.576138
 18197  3.596389
 18198  3.498395
 18199  5.501227
 18200  3.548837
 
 [18201 rows x 1 columns])

In [24]:
#initializing the artificial neural network
import tensorflow as tf
from keras.layers import Dense
ann=tf.keras.models.Sequential()
from keras.layers import Dense
#Creating hidden layers
ann.add(Dense(units=40,activation="relu"))
ann.add(Dense(units=30,activation="relu"))
ann.add(Dense(units=20,activation="relu"))
ann.add(Dense(units=10,activation="relu"))
ann.add(Dense(units=5,activation="relu"))
ann.add(Dense(units=1,activation="sigmoid"))#output layer
ann.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.009),loss="mse",metrics=['mean_absolute_error'])#compile the ANN

In [25]:
#fitting the ANN
ann.fit(X_train,d,batch_size=64,epochs = 10)

Epoch 1/10
1138/1138 [==============================] - 5s 3ms/step - loss: 11.7524 - mean_absolute_error: 2.9475
Epoch 2/10
1138/1138 [==============================] - 3s 3ms/step - loss: 11.7513 - mean_absolute_error: 2.9473
Epoch 3/10
1138/1138 [==============================] - 4s 3ms/step - loss: 11.7513 - mean_absolute_error: 2.9473
Epoch 4/10
1138/1138 [==============================] - 4s 3ms/step - loss: 11.7513 - mean_absolute_error: 2.9473
Epoch 5/10
1138/1138 [==============================] - 5s 4ms/step - loss: 11.7513 - mean_absolute_error: 2.9473
Epoch 6/10
1138/1138 [==============================] - 3s 3ms/step - loss: 11.7513 - mean_absolute_error: 2.9473
Epoch 7/10
1138/1138 [==============================] - 4s 3ms/step - loss: 11.7513 - mean_absolute_error: 2.9473
Epoch 8/10
1138/1138 [==============================] - 4s 3ms/step - loss: 11.7513 - mean_absolute_error: 2.9473
Epoch 9/10
1138/1138 [==============================] - 4s 4ms/step - loss: 11.7513 - me

In [26]:
print(ann.evaluate(X_test,Y_test))


569/569 [==============================] - 2s 2ms/step - loss: 11.6857 - mean_absolute_error: 2.9426
[11.685698509216309, 2.9425768852233887]


In [28]:
pred=pd.DataFrame(ann.predict(X_test))
print(pred)

569/569 [==============================] - 1s 2ms/step
         0
0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
...    ...
18196  1.0
18197  1.0
18198  1.0
18199  1.0
18200  1.0

[18201 rows x 1 columns]
